<a href="https://colab.research.google.com/github/Chirag314/Cancerdata-ensemble-model/blob/main/Cancerdata_Ensemble_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This notebook is copied from exercises from book Ensemble Machine Learning Cookbook.

In [1]:
# Read data from github. Use raw format and copy url# Note normal url and raw url will be different.
import pandas as pd
pd.options.display.max_rows=None
pd.options.display.max_columns=None
url = 'https://raw.githubusercontent.com/PacktPublishing/Ensemble-Machine-Learning-Cookbook/master/Chapter02/wisc_bc_data.csv'
df_cancerdata = pd.read_csv(url)
#df = pd.read_csv(url)
print(df_cancerdata.head(5))

         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0  87139402         B        12.32         12.39           78.85      464.1   
1   8910251         B        10.60         18.95           69.28      346.4   
2    905520         B        11.04         16.83           70.92      373.2   
3    868871         B        11.28         13.39           73.00      384.8   
4   9012568         B        15.19         13.21           97.65      711.8   

   smoothness_mean  compactness_mean  concavity_mean  points_mean  \
0          0.10280           0.06981         0.03987      0.03700   
1          0.09688           0.11470         0.06387      0.02642   
2          0.10770           0.07804         0.03046      0.02480   
3          0.11640           0.11360         0.04635      0.04796   
4          0.07963           0.06934         0.03393      0.02657   

   symmetry_mean  dimension_mean  radius_se  texture_se  perimeter_se  \
0         0.1959         0.05955     

#####Here, we have a dataset based on the properties of cancerous tumors. Using this dataset, we'll build multiple classification models with diagnosis as our response variable. The diagnosis variable has the values, B and M, which indicate whether the tumor is benign or malignant. With multiple learners, we extract multiple predictions. The weighted averaging technique takes the average of all of the predicted values for each training sample.

In [2]:
#import required libraries
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier


In [3]:
#Create train and test cample from our dataset
from sklearn.model_selection import train_test_split

# Create feature and response state
#feature_columns=['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide', 'total sulfur dioxide','density', 'pH','sulphates', 'alcohol']
X=df_cancerdata.iloc[:,2:32]
y=df_cancerdata['diagnosis']

# Create train and test results
X_train, X_test, Y_train, Y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [4]:
# create the sub models
estimators = []

dt_model = DecisionTreeClassifier()
estimators.append(('DecisionTree', dt_model))

svm_model = SVC(probability=True)
estimators.append(('SupportVector', svm_model))

logit_model = LogisticRegression()
estimators.append(('Logistic Regression', logit_model))

In [5]:
dt_model.fit(X_train, Y_train)
svm_model.fit(X_train, Y_train)
logit_model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [6]:
#Use the predict_proba() function to predict the class probabilities:
dt_predictions=dt_model.predict_proba(X_test)
svm_predictions=svm_model.predict_proba(X_test)
logit_predictions=logit_model.predict_proba(X_test)

In [13]:
#Assign different weights to each of the models to get our final predictions:
weighted_average_predictions=(dt_predictions * 0.3 + svm_predictions * 0.4 + logit_predictions * 0.3)
#print(weighted_average_predictions)

In [14]:
# Find the accuracy metrics
from sklearn import metrics
#MAE
mae_linreg=metrics.mean_absolute_error(weighted_average_predictions,Y_test)
#mae_svc=metrics.mean_absolute_error(svr_predictions,Y_test)
#mae_tree=metrics.mean_absolute_error(regtree_predictions,Y_test)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE of linear regression is {}'.format(mae_linreg))
#print('MAE of svc is {}'.format(mae_svc))
#print('MAE of decision tree is {}'.format(mae_tree))

ValueError: ignored